In [ ]:
import numpy as np
import pandas as pd

# np.zeros((5,4))
# np.random.randint(1,7,(5,4))
df = pd.DataFrame(np.random.randint(0,100,size=(3, 4)), columns=list('ABCD'))
df.iloc[0] = [np.nan, np.nan, np.nan, 'nan']
print(df.isnull().sum().sum())
df


In [ ]:
pd.concat([df, pd.DataFrame(np.array([1,2,3]))], axis=1, ignore_index=True)

In [ ]:
from collections import Counter

c = Counter([1,2,3])
c[4]

In [ ]:
import sys
-sys.maxsize

In [ ]:
#Datetime functions

# import pandas as pd
# import datetime
# from dateutil.relativedelta import relativedelta

# class Const:
#     cur_date = datetime.datetime.today()
#     prior_date = (datetime.datetime.today() + relativedelta(months=-1))
#     date_ranges = pd.date_range(start=prior_date.strftime("%m/%d/%Y"), end=cur_date.strftime("%m/%d/%Y"))

import io
data = {'date':['03/28/2019  10:08:44 PM','4/26/2019  12:43:07 AM','4/35/2019  12:43:07 AM','',0,'0']}
df = pd.DataFrame(data)
# df = pd.read_csv(io.StringIO(data))
# encoding='latin1')

df['f_date'] = pd.to_datetime(df['date'], errors='coerce')

max_date = max(df['f_date'])
date_val = max_date - pd.to_timedelta(50, 'D') 
df['flag'] = (df['f_date'] >= date_val)

print(df)
print(max_date, date_val)

## Multiprocessing

In [ ]:
# https://www.journaldev.com/15631/python-multiprocessing-example

## Bash

In [ ]:
# export PS1='\!:\[\033[0;32m\]$( basename `git rev-parse --show-toplevel 2> /dev/null` 2> /dev/null | sed "s/^/[/")$( git branch 2> /dev/null | cut -f2 -d\* -s | sed "s/^ /:/" | sed "s/$/]/" )\[\033[0m\]$( pwd | sed "s|`git rev-parse --show-toplevel 2> /dev/null`||" 2> /dev/null || dirs +0) \$ '


## Cloud

In [ ]:
#Read many part files from AWS s3

import boto3, io, s3fs
from multiprocessing import Pool

## Connections

# conn = connect(
#     aws_access_key_id='YOUR_ACCESS_KEY_ID',
#     aws_secret_access_key='YOUR_SECRET_ACCESS_KEY',
#     s3_staging_dir='s3://bucket/staging/',
#     region_name='us-east-1')

# cursor = connect(
#     aws_access_key_id='YOUR_ACCESS_KEY_ID',
#     aws_secret_access_key='YOUR_SECRET_ACCESS_KEY',
#     s3_staging_dir='s3://bucket/staging/',
#     region_name='us-east-1').cursor()

## --------------------------------------------------
## Pandas batch style based data fetch (Error: Query times out for large datasets)

# def read_data_from_athena(db_name, table_name):
#     query = "SELECT * FROM {}.{}".format(db_name, table_name) 
#     cur_df = pd.read_sql(query, conn)
#     return cur_df

# raw_training_df = read_data_from_athena(db_name, table_name)

## --------------------------------------------------
## Cursor based data fetch (Error: No result set for large datasets)

# def get_cursor_iterator(db_name, table_name):
#     query = "SELECT * FROM {}.{} limit 10".format(db_name, table_name)
#     print(query)
#     for row in cursor:
#         print(row)
#         break

# get_cursor_iterator(db_name, table_name)

## --------------------------------------------------
def serial_read_part_files_to_df(bucket_name, parent_key):
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucket_name)
    prefix_objs = bucket.objects.filter(Prefix=parent_key)
    sub_df = []
    for obj in prefix_objs:
        file_key = obj.key
        print(file_key)
        cur_df = pd.read_csv('s3://'+bucket_name+'/'+file_key, compression='gzip', header=0, sep='|', quotechar='"')
        cur_df = cur_df[valid_cols]
        sub_df.append(cur_df)
    return pd.concat(sub_df)

def get_data(bucket_name, file_key, valid_cols):
    print(file_key)
    cur_df = pd.read_csv('s3://'+bucket_name+'/'+file_key, compression='gzip', header=0, sep='|', quotechar='"')
    cur_df = cur_df[valid_cols]
    return cur_df

class GET_DATA(): #Helper class to invoke get_data() as local lambda's cannot be pickled so cannot be used with pool.map. functools.partial can also be used 
    def __init__(self, bucket_name, valid_cols):
        self.bucket_name = bucket_name
        self.valid_cols = valid_cols
    def __call__(self, file_key):
        return get_data(self.bucket_name, file_key, self.valid_cols)

def parallel_read_part_files_to_df(bucket_name, parent_key):
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucket_name)
    prefix_objs = bucket.objects.filter(Prefix=parent_key)
    file_keys = tuple(map(lambda x : x.key, prefix_objs))
    agents, chunksize = 5, 25
    GD = GET_DATA(bucket_name, valid_cols)
    with Pool(processes=agents) as pool:
        sub_df = pool.map(GD, file_keys, chunksize)
    return pd.concat(sub_df)


## Monitoring

In [ ]:
##Publish metrics to cloudwatch metrics
# https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/cloudwatch_concepts.html

d = {1:2, 3:4, 5:6}

import boto3
cloudwatch = boto3.client('cloudwatch')

def is_potential_float(v):
    try:
        float(v)
        return True
    except:
        return False
    
#Rather than differentiating dev/prod metrics by dimensions lets push dev/prod metrics to different namespaces
def iterate_dict(d, cur_metric_data):
    for k,v in d.items():
        if isinstance(v, dict):
            iterate_dict(v, cur_metric_data)
        elif is_potential_float(v):
            v = float(v)
            cur_metric_data.append(
                {
                    'MetricName': k,
                    'Unit': 'None',
                    'Value': v
                }
            )

def publish_cw_metricdata_from_dict(d):
    cur_metric_data = []
    cur_namespace = 'a/b'
    iterate_dict(d, cur_metric_data)
    #print(cur_metric_data)
    response = cloudwatch.put_metric_data(
        MetricData = cur_metric_data,
        Namespace = cur_namespace
    )
    return response

response = publish_cw_metricdata_from_dict(d)
response


In [ ]:
##Publish metrics to Prometheus
# https://github.com/prometheus/client_python

##Publish metrics to elastic search
# https://tryolabs.com/blog/2015/02/17/python-elasticsearch-first-steps/


## Docker

## Gitlab CI/CD

## VirtualEnv